# Préambule

Ce notebook correspond à la base de données Neo4j réalisée dans le cadre du master EBAM 2019 - 2020 par Anthony Moisan et Léonard Péan

Prérequis, il est nécessaire :
1. de lancer en local un serveur Neo4j, créer une base de données (user par défaut : neo4j et mot de passe : password sinon il faudra adapter le script), de lancer la base de données 
2. d'installer le plugin APOC via Manage -> Plugins -> APOC

# Small Example

Petit jeu de données créés à la main pour voir la pertinence des requêtes. Niveau1_Small et Niveau2_Small. Les fichiers CSV doivent être situés dans le fichier d'import de la base de données via Manage -> Open Folders -> Import

## Packages

Installation des packages nécessaires à l'execution py2neo pour la connexion à Python vers Neo4j et pandas pour la visualisation des résultats à travers des dataframe

In [7]:
!pip install py2neo
!pip install pandas
import pandas as pd

## Connexion à la base de données

In [8]:
# les trois premiers éléments sont à configurer en fonction de la création en local de la base de données Neo4j
from py2neo import Graph
myUri = "http://localhost:7474"
myUser = "neo4j"
myPassword = "password"
graph = Graph(myUri, username=myUser, password=myPassword)

## Ensemble des procédures pour créer le graphe ou le détruire

In [9]:
#Permet de mettre les contraintes d'intégrité et index
def DefineConstraintAndIndex(db):
    queryInsertConstraintEntityCodeLEI = "CREATE CONSTRAINT ON (e:Entity) ASSERT e.code_LEI IS UNIQUE;"
    db.run(queryInsertConstraintEntityCodeLEI)
    queryInsertIndexCountryName = "CREATE INDEX ON :Country(name);"
    db.run(queryInsertIndexCountryName)
    queryInsertIndexEntiteName = "CREATE INDEX ON :Entity(name);"
    db.run(queryInsertIndexEntiteName)
    queryDropIndexActiveName = "CREATE INDEX ON :Active(name);"
    db.run(queryDropIndexActiveName)


In [10]:
#Permet de définir l'import de Niveau 1
def DefineImportNiveau1(db,namefile):
    queryLevel1 ="USING PERIODIC COMMIT 500 " \
    "LOAD CSV WITH HEADERS FROM 'file:///"+namefile+".csv' " \
    "AS ligne FIELDTERMINATOR ';' " \
    "MERGE (e:Entity {code_LEI: ligne.Code_LEI, name : ligne.Legal_Name, last_update : apoc.date.format(apoc.date.parse(ligne.Last_Update_Date, 'ms', 'yyyy-MM-dd'))}) "  \
    "MERGE (act:Active {name:ligne.Status_Actif}) " \
    "MERGE (country:Country {name: ligne.Country_Code}) " \
    "MERGE (e)-[:ACTIVE]-(act) " \
    "MERGE (e)-[:REGISTERED]-(country)" 
    db.run(queryLevel1)

In [11]:
#Permet de définir l'import de Niveau 2
def DefineImportNiveau2(db,namefile):
    queryLevel2 ="USING PERIODIC COMMIT 500 " \
    "LOAD CSV WITH HEADERS FROM 'file:///"+namefile+".csv' " \
    "AS ligne FIELDTERMINATOR ';' " \
    "MATCH (startNode:Entity{ code_LEI: ligne.Start_Node}),(endNode:Entity { code_LEI: ligne.End_Node}) " \
    "MERGE (startNode)-[:RELATIONSHIP { role: ligne.Relation_ShipeType }]->(endNode)"
    db.run(queryLevel2)

In [12]:
#1) On créer les contraintes et index
DefineConstraintAndIndex(graph)
#2) On réalise l'import de niveau 1 pour initialiser une partie du graphe
DefineImportNiveau1(graph,'Niveau1_Small')
#3) On réalise l'import de niveau 2 pour mettre les relations entre les entités juridiques
DefineImportNiveau2(graph,'Niveau2_Small')

![schema_entite](../img/CasJouetEntite.png)

On a créé fictivement à travers ce petit jeu de données deux holding (SMA et AXA). Le graph avec les noms n'apparait pas explicitement mais peut être vue à travers l'éditeur Neo4j. La profondeur d'Axa est plus important dans notre cas jouet.

![active](../img/ActiveCasJouet.png)

Un autre étiquette en plus des entités est le fait que la filiale est active ou non car la base de données enregistre dans le temps les schémas de consolidation et que la vie des entreprises est faite d'acquisitions et de cessions. Dans notre cas jouet, 4 entités sont considérées comme inactive.

![pays](../img/PaysCasJouet.png)

La notion de pays associé à une entité a aussi été insérée comme étiquette. Dans notre cas jouet, nous avons 5 pays où les entités juridiques sont enregistrées. 

In [13]:
#Permet de supprimer l'ensemble des noeuds et des relations 
def Armageddon(db):
    queryDeleteNodesAndRelations = "MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r"
    db.run(queryDeleteNodesAndRelations)    

In [14]:
#Permet de supprimer les contraintes d'intégrité et les index
def DropConstraintAndIndex(db):
    queryDropIndexCountryName = "DROP INDEX ON :Country(name);"
    db.run(queryDropIndexCountryName)
    queryDropIndexEntiteName = "DROP INDEX ON :Entity(name);"
    db.run(queryDropIndexEntiteName)
    queryDropIndexActiveName = "DROP INDEX ON :Active(name);"
    db.run(queryDropIndexActiveName)
    queryDropConstraintEntityCodeLEI = "DROP CONSTRAINT ON (e:Entity) ASSERT e.code_LEI IS UNIQUE;"
    db.run(queryDropConstraintEntityCodeLEI)

## Ensemble des requêtes

### Nombre de noeuds et relations

In [40]:
def VisualizeGraph(db) :
    qCountEntityActive = "MATCH (e:Entity)-[:ACTIVE]-(act) " \
            "WHERE act.name = 'ACTIVE' " \
            "RETURN count(e) "
    numberOfActiveEntity = db.run(qCountEntityActive).evaluate()
    
    qCountEntity = "MATCH (e:Entity) " \
                "RETURN count(e) "
    numberOfEntity = db.run(qCountEntity).evaluate()
    
    qCountCountry = "MATCH (c:Country) " \
                "RETURN count(c) "
    numberOfCountry = db.run(qCountCountry).evaluate()
    
    qCountActive = "MATCH (a:Active) " \
                "RETURN count(a) "
    numberOfActive = db.run(qCountActive).evaluate()
    
    
    qRelationShip ="MATCH p=()-[r:RELATIONSHIP]->() RETURN count(p)"
    numberOfRelationShip = db.run(qRelationShip).evaluate()
    
    qRelationActive ="MATCH p=()-[r:ACTIVE]->() RETURN count(p)"
    numberOfRelationActive = db.run(qRelationActive).evaluate()
    
    qRelationRegistered ="MATCH p=()-[r:REGISTERED]->() RETURN count(p)"
    numberOfRelationRegistered = db.run(qRelationRegistered).evaluate()
    
    print("--- About Nodes ")
    print("* Number of active entity  : " + str(numberOfActiveEntity))
    print("* Number of total entity  : " + str(numberOfEntity))
    print("* % of active entity : " + str(round(numberOfActiveEntity/numberOfEntity*100,2)))
    print("* Number of country  : " + str(numberOfCountry))
    print("* Number of active  : " + str(numberOfActive))
    
    print("\n--- About Relations ")
    print("* Number of relationship between Entities : " + str(numberOfRelationShip))
    print("* Number of relationship between Entitie/Active : " + str(numberOfRelationActive))
    print("* Number of relationship between Entitie/Country : " + str(numberOfRelationRegistered))
    

In [36]:
VisualizeGraph(graph)

--- About Nodes 
* Number of active entity  : 73332
* Number of total entity  : 67187
* % of active entity : 91.62
* Number of country  : 17
* Number of active  : 2

--- About Relations 
* Number of relationship between Entities : 11259
* Number of relationship between Entitie/Active : 73332
* Number of relationship between Entitie/Country : 73332


On retrouve bien dans notre cas jouet :
* 17 noeuds : 10 pour les entités, 5 pour les pays, 2 pour le caractère actif ou inactif
* 28 relation : 8 de liens entre les entités, 10 pour le caractère actif et 10 pour la domiciliation

### Nombre d'entités légales par pays par ordre croissant

#### 1ère requête sans prise en compte du caractère active ou non de l'entité

In [17]:
def EntityPerCountry(db) :
    query = "MATCH (c:Country)-[:REGISTERED]-(e:Entity) " \
    "RETURN distinct(c.name) as Country, count(e) as Number ORDER BY Number DESC"
    return pd.DataFrame(db.run(query).data())

In [18]:
EntityPerCountry(graph)

,Country,Number
0,FR,6
1,IT,1
2,ES,1
3,LU,1
4,BS,1


Cela correspond aux nationalités des entités fournies dans le cas jouet

#### 2ème requête avec prise en compte du caractère active ou non de la filiale


In [19]:
def EntityPerCountryActive(db) :
    query = "MATCH (c:Country)-[:REGISTERED]-(e:Entity)-[:ACTIVE]-(act) " \
    "WHERE act.name = 'ACTIVE' " \
    "RETURN distinct(c.name) as Country, count(e) as Number ORDER BY Number DESC"
    return pd.DataFrame(db.run(query).data())

In [20]:
EntityPerCountryActive(graph)

,Country,Number
0,FR,3
1,IT,1
2,ES,1
3,LU,1


Cela correspond aux nationalités des entités actives dans le cas jouet.

### Schema consolidation AXA

#### 1ère requête sans prise en compte du caractère active ou non de la filiale

On cherche à partir d'une société à avoir toutes les filiales qui en dépendent

In [21]:
def SchemaConsolidationAxa(db,society) :
    query = "MATCH (e:Entity {name:"+society+"}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsdiary:Entity) " \
    "return extract(x IN nodes(path) | x.name) as Nodes,length(path) as Depth ORDER BY Depth ASC"
    return pd.DataFrame(db.run(query).data())


In [22]:
SchemaConsolidationAxa(graph, "'AXA'")

,Depth,Nodes
0,1,"[AXA, AXA_VIE]"
1,1,"[AXA, AXA_IARD]"
2,2,"[AXA, AXA_IARD, AXA_PICARD]"
3,2,"[AXA, AXA_IARD, AXA_Finance]"
4,3,"[AXA, AXA_IARD, AXA_PICARD, AXA_PICARD_OUEST]"
5,3,"[AXA, AXA_IARD, AXA_PICARD, AXA_PICARD_SUD]"


Schéma cohérent avec Axa en prenant en compte dans le cas présent les filiales inactives

#### 2ème requête en prenant le caractère active des filiales

In [23]:
def SchemaConsolidationWithActiveSubsidiary(db, society) :
    query = "MATCH (e:Entity {name:" + society + "}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsidiary:Entity)-[:ACTIVE]-(act {name:'ACTIVE'}) " \
    "RETURN extract(x IN nodes(path) WHERE x.name <> 'ACTIVE' | x.name) as Nodes,length(path) as Depth ORDER BY Depth ASC"
    return pd.DataFrame(db.run(query).data())

In [24]:
SchemaConsolidationWithActiveSubsidiary(graph, "'AXA'")

,Depth,Nodes
0,2,"[AXA, AXA_VIE]"
1,2,"[AXA, AXA_IARD]"
2,3,"[AXA, AXA_IARD, AXA_Finance]"


Schéma cohérent avec les filiales actives d'Axa. Toutes les filiales dépendant de AXA_PICARD ne sont plus visibles car inactives

### Calcul de profondeur entre société mère et filiales

In [25]:
#Permet de savoir si une société est la société mère (elle n'a pas de relations où elle est le noeud de sortie dans une relation de filiation avec d'autres entités juridiques)
#On ajoute un champ au noeud entité IsParent
def DefineHolding(db) :
    queryIsParentFalse = "MATCH (e:Entity)<-[RELATIONSHIP]-() " \
    "SET e.isParent = FALSE " \
    "RETURN e"
    db.run(queryIsParentFalse)
    
    queryIsParentTrue = "MATCH (e:Entity) " \
    "WHERE not exists(e.isParent) " \
    "SET e.isParent = TRUE " \
    "return e"
    db.run(queryIsParentTrue)

In [26]:
DefineHolding(graph)

In [27]:
#Permet de calculer la profondeur entre les sociétés mères d'un pays et ses filiales
def CalculateHoldingSubsidiaryPerCountry(db, nameCountry):
    query = "MATCH (c:Country {name:"+nameCountry+"})-[:REGISTERED]-(e:Entity {isParent:TRUE})-[:ACTIVE]-(act{name:'ACTIVE'}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsidiary:Entity)-[:ACTIVE]-(act{name:'ACTIVE'}) " \
    "return e.name as Holding, subsidiary.name as Subsidiary, length(path) as Depth ORDER BY Depth DESC "
    return pd.DataFrame(db.run(query).data())

In [28]:
CalculateHoldingSubsidiaryPerCountry(graph, "'FR'")

,Depth,Holding,Subsidiary
0,3,AXA,AXA_Finance
1,2,AXA,AXA_VIE
2,2,AXA,AXA_IARD
3,2,SMA,MUTUA_ESPAGNA


Les résultats correspondent aux filiales uniquement actives d'Axa et de SMA pour les entités enregistrées en France.

### Identification de l'évasion fiscale potentielle

In [29]:
#Permet de voir les sociétés mères d'un pays qui ont des filiales actives dans des paradis fiscaux (listCountrySuspect)
def IdentifySuspectSubsidiaries(db,nameCountry, listCountrySuspect):
    query = "MATCH (c:Country {name:"+nameCountry+"})-[:REGISTERED]-(e:Entity {isParent:TRUE})-[:ACTIVE]-(act{name:'ACTIVE'}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsidiary:Entity)-[:REGISTERED]-(cSubsidiary:Country) WHERE cSubsidiary.name in "+listCountrySuspect+ "AND (subsidiary:Entity)-[:ACTIVE]-(act{name:'ACTIVE'})" \
    "return e.name as Entity, subsidiary.name as Subsidiary, length(path) as Depth"
    return pd.DataFrame(db.run(query).data())

In [30]:
listCountrySuspect = "['LU', 'BS']" #permet d'identifier le luxembourg et Les iles Bahamas
IdentifySuspectSubsidiaries(graph,"'FR'", listCountrySuspect )

,Depth,Entity,Subsidiary
0,3,AXA,AXA_Finance


L'administration fiscale peut rechercher les filiales actives dépedant d'une holding française qui sont dans des paradis fiscaux identifiés dans le cas présent par une domiciliation au Luxembourg et au Bahamas. Dans le cas présent, le résultat est cohérent car il ne fait pas apparaître la filiale non active dans les Bahamas associée à l'entité SMA et on retrouve AXA_Finance qui est domicilié au luxembourg 

## Destruction du graph

In [31]:
Armageddon(graph)
DropConstraintAndIndex(graph)

# Jeu de données réelles
Les jeux de données réelles sont basées sur la transformation XML des données accessibles en Open Data sur le site https://www.gleif.org via un CSV. Le code Python permet de faire la conversion se trouve aussi dans un autre projet Python dans SRC

## Transformation des données

Un convertisseur XML vers CSV a été réalisé dans le cadre de ce projet avec deux fonctions pour convertir :
* les fichiers de niveau 1
* les fichiers de niveau 2

**Attention :** il est nécessaire de télécharger les fichiers XML du gleif [télécharger](https://www.gleif.org/fr/lei-data/gleif-concatenated-file/download-the-concatenated-file) dans le répertoire xml de l'arborescence projet. Il n'était pas possible de les mettre sous Github du fait de la volumétrie. 

Nous avons renommé par commodité le fichier en niveau1.xml et en niveau2.xml après les avoir dezippé dans le répertoire xml.

Nous avons mis le résultat de la conversion sur la base GLEIF de l'insee dans data. Aussi, vous pouvez passer cette étape de conversion des données en allant directement à l'étape de création du Graph.

In [ ]:
from utils.ConvertGLEIF import ConvertFichierNiveau1
from utils.ConvertGLEIF import ConvertFichierNiveau2

In [ ]:
#Permet de convertir le fichier XML de niveau 1 et récupère le dictionnaire des codes LEI pour cohérence avec fichier niveau 2
dictCodeLEI = ConvertFichierNiveau1("../xml/niveau1.xml","../data/Niveau1_Real.csv")
#Permet de convertir le fichier XML de niveau 2 en s'assurant que les données correspondent au code LEI du fichier de niveau 1
ConvertFichierNiveau2("../xml/niveau2.xml","../data/Niveau2_Real.csv", dictCodeLEI)

## Creation du graph

### Modification de la requete de creation des entités

Mon script d'import des entités sur un jeu de données de plus de 70000 lignes mettait 15 mn ce qui me paraissait vraiment trop long.

Mon script initial était le suivant :

> script initial

`
USING PERIODIC COMMIT 500 

LOAD CSV WITH HEADERS FROM 'file:///Niveau1_Real.csv' 
AS ligne FIELDTERMINATOR ';' 

   MERGE (e:Entity {code_LEI: ligne.Code_LEI})
   SET e.name = ligne.Legal_Name
   SET e.last_update = apoc.date.format(apoc.date.parse(ligne.Last_Update_Date, 'ms', 'dd/MM/yyyy'))

   MERGE (act:Active {name:ligne.Status_Actif})

   MERGE (country:Country {name: ligne.Country_Code})

MERGE (e)-[:ACTIVE]-(act) 

MERGE (e)-[:REGISTERED]-(country)
`

> script modifié

`
USING PERIODIC COMMIT 500 

LOAD CSV WITH HEADERS FROM 'file:///Niveau1_Real.csv' 
AS ligne FIELDTERMINATOR ';' 

   MERGE (e:Entity {code_LEI: ligne.Code_LEI, name : ligne.Legal_Name, last_update : apoc.date.format(apoc.date.parse(ligne.Last_Update_Date, 'ms', 'dd/MM/yyyy'))})

   MERGE (act:Active {name:ligne.Status_Actif})

   MERGE (country:Country {name: ligne.Country_Code})

MERGE (e)-[:ACTIVE]-(act) 

MERGE (e)-[:REGISTERED]-(country)
`

Le temps est de l'ordre de quelques secondes. Nous avons ne pas comprendre le comportement différent entre les deux scripts

In [32]:
import time
startTime = time.time()
#1) On créer les contraintes et index
DefineConstraintAndIndex(graph)
#2) On réalise l'import de niveau 1 pour initialiser une partie du graphe
DefineImportNiveau1(graph,'Niveau1_Real')
#3) On réalise l'import de niveau 2 pour mettre les relations entre les entités juridiques
DefineImportNiveau2(graph,'Niveau2_Real')
endTime = time.time()
print ("Temps pour la création du graphe %f" %(endTime-startTime))

Temps pour la création du graphe 9.827404


### Nombre de noeuds et relations

In [41]:
VisualizeGraph(graph)

--- About Nodes 
* Number of active entity  : 67187
* Number of total entity  : 73332
* % of active entity : 91.62
* Number of country  : 17
* Number of active  : 2

--- About Relations 
* Number of relationship between Entities : 11259
* Number of relationship between Entitie/Active : 73332
* Number of relationship between Entitie/Country : 73332


### Nombre d'entités légales par pays par ordre croissant

#### 1ère requête sans prise en compte du caractère active ou non de l'entité

In [38]:
EntityPerCountry(graph)

,Country,Number
0,FR,73267
1,LU,25
2,GB,16
3,DE,4
4,IT,4
5,PF,2
6,BE,2
7,NL,2
8,NC,2
9,IE,1


#### 2ème requête avec prise en compte du caractère active ou non de la filiale

In [42]:
EntityPerCountryActive(graph)

,Country,Number
0,FR,67143
1,GB,13
2,LU,13
3,DE,3
4,NL,2
5,BE,2
6,PF,2
7,NC,2
8,CH,1
9,SE,1


### Schema consolidation Société X

In [ ]:
SchemaConsolidationWithActiveSubsidiary(graph, "'AXA'")
#Code_LEI F5WCUMTUM4RKZ1MAIE39

### Calcul de profondeur entre société mère et filiales

In [ ]:
DefineHolding(graph)
CalculateHoldingSubsidiaryPerCountry(graph, "'FR'")

### Identification de l'évasion fiscale potentielle

In [ ]:
listCountrySuspect = "['LU', 'BS']" #permet d'identifier le luxembourg et Les iles Bahamas
IdentifySuspectSubsidiaries(graph,"'FR'", listCountrySuspect )

## Destruction du graph

In [ ]:
Armageddon(graph)
DropConstraintAndIndex(graph)

# Conclusion

Dans le cadre de ce projet, nous avons pu :
* manipuler une base de données Neo4j de sa création, avec son langage spécifique de requêtes, jusqu'à sa suppression
* voir la force par rapport à une base de données relationnelles pour identifier les relations d'amis / d'amis..., qui est un vrai point fort de Neo4j 
* améliorer nos compétences en Python pour réaliser les transformations de données et appeler directement Neo4j/Cypher via py2neo. 

Les difficultés auxquelles nous avons été confrontées :
* la difficulté lorsque Cypher ne renvoie rien pour identifier l'erreur dans les requêtes
* le coût d'un langage qui n'est pas évident par exemple dans la prise en comptes des dates dans les propriétés et de la librairie py2neo qui s'est avérée plus simple par rapport à l'API Rest
